<a href="https://colab.research.google.com/github/juliocesaria/emotion-detection-nlp/blob/main/02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Preprocessing notebook initialized")


Preprocessing notebook initialized
